In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import pip

In [6]:
Customer_Demo = pd.read_excel('CustomerDemographics.xlsx')
Customer_Behavior_Data =  pd.read_excel('CustomerBehaviorData.xlsx')
Credit_Consumption_Data = pd.read_excel('CreditConsumptionData.xlsx')

In [7]:
# Get metadata using info() method
print("DataFrame info:")
print(Customer_Demo.info())

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            20000 non-null  int64  
 1   account_type                  19999 non-null  object 
 2   gender                        19999 non-null  object 
 3   age                           20000 non-null  int64  
 4   Income                        19999 non-null  object 
 5   Emp_Tenure_Years              20000 non-null  float64
 6   Tenure_with_Bank              20000 non-null  int64  
 7   region_code                   19999 non-null  float64
 8   NetBanking_Flag               20000 non-null  int64  
 9   Avg_days_between_transaction  19997 non-null  float64
dtypes: float64(3), int64(4), object(3)
memory usage: 1.5+ MB
None


In [8]:
# Get summary statistics using describe() method
print("DataFrame summary statistics:")
print(Customer_Demo.describe())

DataFrame summary statistics:
                 ID           age  Emp_Tenure_Years  Tenure_with_Bank  \
count  20000.000000  20000.000000      20000.000000      20000.000000   
mean    9999.500000     47.672300         15.217250          5.496050   
std     5773.647028     30.359771         20.481455          2.870452   
min        0.000000     25.000000          0.500000          1.000000   
25%     4999.750000     33.000000          5.100000          3.000000   
50%     9999.500000     40.000000          9.900000          6.000000   
75%    14999.250000     53.000000         18.000000          8.000000   
max    19999.000000    223.000000        203.000000         10.000000   

        region_code  NetBanking_Flag  Avg_days_between_transaction  
count  19999.000000     20000.000000                  19997.000000  
mean     598.959348         0.718000                     11.044757  
std      220.105230         0.449985                      5.480552  
min      123.000000         0.000000

In [9]:
# Get metadata using info() method
print("DataFrame info:")
print(Customer_Behavior_Data.info())

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     20000 non-null  int64  
 1   cc_cons_apr            20000 non-null  float64
 2   dc_cons_apr            20000 non-null  float64
 3   cc_cons_may            19999 non-null  float64
 4   dc_cons_may            20000 non-null  float64
 5   cc_cons_jun            20000 non-null  float64
 6   dc_cons_jun            19999 non-null  float64
 7   cc_count_apr           19999 non-null  float64
 8   cc_count_may           20000 non-null  int64  
 9   cc_count_jun           20000 non-null  int64  
 10  dc_count_apr           20000 non-null  int64  
 11  dc_count_may           20000 non-null  int64  
 12  dc_count_jun           20000 non-null  int64  
 13  card_lim               20000 non-null  int64  
 14  personal_loan_active   20000 non-null 